In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns

import string
import sqlite3

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
from sklearn.metrics import confusion_matrix, auc, roc_curve
from nltk.stem.porter import PorterStemmer

%matplotlib inline

In [3]:
conn = sqlite3.connect("datasets/food_reviews.sqlite")

In [4]:
filtered_data = pd.read_sql_query("""
    select * from Reviews
    WHERE Score != 3
""",conn)

In [5]:
actualScores = filtered_data['Score']
posNeg = actualScores.map(lambda i:"Positive" if i>3 else "Negative")
filtered_data["Score"] = posNeg

In [6]:
print(filtered_data.shape)
filtered_data.head(3)

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


## Data Cleaning : DeDuplication
We notice that for same user : Time, Summary, Text are almost same
Reason : Different variations of same product

In [8]:
display = pd.read_sql_query("""
select * from Reviews 
where Score !=3 and UserId = "AR5J8UI46CURR"
ORDER by ProductID
""",conn)

display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [10]:
#Sort data acc o ProductID
sorted_data = filtered_data.sort_values('ProductId', axis=0,ascending = True)

In [12]:
#deduplication
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

Previous Data  : 525814 <br>
After Deduplication : 364173

### Another Problem : HelpNum and HelpDenom 
num : ppl who said yes<br>
denom : all ppl <br>
In some cases num > denom

In [18]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final['Score'].value_counts()

Positive    307061
Negative     57110
Name: Score, dtype: int64

## TEXT PROCESSING : Stemming, Stop Word Removal, Lemmetization